In [1]:
from tensorflow.keras.datasets import mnist
import tensorflow as tf
import numpy as np
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
X_train = tf.placeholder(tf.float32, shape=[None, 28, 28, 1]) #输入
Y_train = tf.placeholder(tf.float32, shape=[None, 10])

In [3]:
Weight1 = tf.get_variable("Weight1", [5, 5, 1, 6] ,initializer = tf.truncated_normal_initializer(stddev=0.1))
bias1 = tf.get_variable('bias1',[1,1,1,6], initializer = tf.truncated_normal_initializer(0.0))
con1 = tf.nn.relu(tf.nn.conv2d(
    X_train,
    filters=Weight1,  # [filter_height, filter_weight, in_channel, out_channels ]
    strides=[1, 1, 1, 1],  #  [1, strides, strides, 1]
    padding='VALID',
    name='Conv1',
) + bias1)

pool1 = tf.nn.max_pool2d(
    con1,
    ksize=2,
    strides=2,
    padding='VALID',
    name='max_pooling1'
)

In [4]:
Weight2 = tf.get_variable("Weight2", [5, 5, 6, 16] ,initializer = tf.truncated_normal_initializer(stddev=0.1))
bias2 = tf.get_variable('bias2',[1,1,1,16], initializer = tf.truncated_normal_initializer(0.0))
con2 = tf.nn.relu(tf.nn.conv2d(
    pool1,
    filters=Weight2,  # [filter_height, filter_weight, in_channel, out_channels ]
    strides=[1, 1, 1, 1],  #  [1, strides, strides, 1]
    padding='VALID',
    name='Conv2'
) + bias2)

pool2 = tf.nn.max_pool2d(
    con2,
    ksize=2,
    strides=2,
    padding='VALID',
    name='max_pooling2'
)

In [5]:
flat = tf.compat.v1.layers.flatten(
    pool2,
    name='Flatten',
    data_format='channels_last'
)

f1 = tf.compat.v1.layers.dense(
    flat,
    units=120,
    activation=tf.nn.relu,
    name='Full1'
)
f2 = tf.compat.v1.layers.dense(
    flat,
    units=84,
    activation=tf.nn.relu,
    name='Full2'
)
f2 = tf.compat.v1.layers.dense(
    flat,
    units=10,
    name='Softmax'
)

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
loss = tf.losses.softmax_cross_entropy(onehot_labels=Y_train, logits=f2)
opt = tf.compat.v1.train.AdamOptimizer(0.001)
accuracy = tf.metrics.accuracy(labels=tf.argmax(Y_train, axis=1), predictions=tf.argmax(f2, axis=1),)[1]
train = opt.minimize(loss)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
def one_hot(labels, nb_classes=None):
    classes = np.unique(labels)
    if nb_classes is None:
        nb_classes = classes.size
    one_hot_labels = np.zeros((labels.shape[0], nb_classes))

    for i, c in enumerate(classes):
        one_hot_labels[labels == c, i] = 1.
    return one_hot_labels

In [8]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.reshape(x_train, [x_train.shape[0],x_train.shape[1],x_train.shape[2],1])  / 255.
y_train = one_hot(y_train,10)
x_test = np.reshape(x_test, [x_test.shape[0],x_test.shape[1],x_test.shape[2],1]) / 255.
y_test = one_hot(y_test,10)

print('训练集数量%d'%x_train.shape[0])
print('测试集数量%d'%x_test.shape[0])
print('每个example的shape',x_train.shape[1:])
print('每个label的shape',y_train.shape[1:])

训练集数量60000
测试集数量10000
每个example的shape (28, 28, 1)
每个label的shape (10,)


In [10]:
with tf.Session() as sess:
    sess.run(tf.group(tf.global_variables_initializer(),tf.local_variables_initializer()))
    for step in range(1000):
        sess.run(train, feed_dict={X_train:x_train, Y_train:y_train})
        if step % 1 ==0:
            print(sess.run([loss,accuracy], feed_dict={X_train:x_train,Y_train:y_train}))

[2.9617498, 0.09915]
[2.7710667, 0.099141665]


KeyboardInterrupt: 